© 2023 Institute for Clinical Evaluative Sciences. All rights reserved.

TERMS OF USE:
##Not for distribution.## This code and data is provided to the user solely for its own non-commercial use by individuals and/or not-for-profit corporations. User shall not distribute without express written permission from the Institute for Clinical Evaluative Sciences.

##Not-for-profit.## This code and data may not be used in connection with profit generating activities.

##No liability.## The Institute for Clinical Evaluative Sciences makes no warranty or representation regarding the fitness, quality or reliability of this code and data.

##No Support.## The Institute for Clinical Evaluative Sciences will not provide any technological, educational or informational support in connection with the use of this code and data.

##Warning.## By receiving this code and data, user accepts these terms, and uses the code and data, solely at its own risk.

In [ ]:
%cd ../../
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)
import seaborn as sns

from src.config import root_path, can_folder, split_date, SCr_rise_threshold
from src.evaluate import EvaluateClf, EvaluateBaselineModel
from src.model import SimpleBaselineModel
from src.prep_data import PrepDataCAN
from src.summarize import data_description_summary, feature_summary
from src.train import Ensembler, LASSOTrainer, PolynomialModelTrainer, Trainer
from src.utility import (
    initialize_folders, load_pickle, 
    get_nunique_categories, get_nmissing, most_common_categories,
    get_hyperparameters
)
from src.visualize import importance_plot, score_plot, subgroup_performance_plot

In [ ]:
processes = 64
target_keyword = 'SCr|dialysis|next'
main_dir = f'{root_path}/projects/{can_folder}'
adverse_event = 'ckd'
output_path = f'{main_dir}/models/{adverse_event.upper()}'
initialize_folders(output_path)

# Prepare Data for Model Training

In [ ]:
prep = PrepDataCAN(adverse_event=adverse_event, target_keyword=target_keyword)
model_data = prep.get_data(include_comorbidity=True, verbose=True)
model_data

In [ ]:
most_common_categories(model_data, catcol='regimen', with_respect_to='patients', top=10)

In [ ]:
sorted(model_data.columns)

In [ ]:
get_nunique_categories(model_data)

In [ ]:
get_nmissing(model_data)

In [ ]:
prep = PrepDataCAN(adverse_event=adverse_event, target_keyword=target_keyword)
model_data = prep.get_data(missing_thresh=80, include_comorbidity=True, verbose=True)
model_data['next_eGFR'].hist(bins=100)
X, Y, tag = prep.split_and_transform_data(model_data, split_date=split_date)
# remove sessions in model_data that were excluded during split_and_transform
model_data = model_data.loc[tag.index]

In [ ]:
prep.get_label_distribution(Y, tag, with_respect_to='sessions')

In [ ]:
prep.get_label_distribution(Y, tag, with_respect_to='patients')

In [ ]:
# Convenience variables
train_mask, valid_mask, test_mask = tag['split'] == 'Train', tag['split'] == 'Valid', tag['split'] == 'Test'
X_train, X_valid, X_test = X[train_mask], X[valid_mask], X[test_mask]
Y_train, Y_valid, Y_test = Y[train_mask], Y[valid_mask], Y[test_mask]

## Study Characteristics

In [ ]:
subgroups = [
    'sex', 'immigration', 'birth_region', 'language', 'income', 'area_density',
    'regimen', 'cancer_type', 'cancer_location', 'target', 'comorbidity', 'dialysis', 'ckd'
]
data_description_summary(
    model_data, Y, tag, save_dir=f'{output_path}/tables', partition_method='cohort', target_event='CKD', 
    subgroups=subgroups
)

## CKD Characteristics

In [ ]:
def get_ckd_count(df, result, key):
    ckd_levels = {'CKD': 60, 'Grade 3b or worse CKD': 45, 'Grade 4 or worse CKD': 30, 'Grade 5 CKD': 15}
    N = df['ikn'].nunique()
    for ckd_level, thresh in ckd_levels.items():
        count = df.query(f'next_eGFR < {thresh}')['ikn'].nunique()
        result[key][ckd_level] = 0 if count < 6 else f'{count} ({count/N:0.02})'
    count = df.query('dialysis')['ikn'].nunique()
    result[key]['Initiation of dialysis'] = 0 if count < 6 else f'{count} ({count/N:0.02})'

from collections import defaultdict
cohort_masks = {'Development': train_mask | valid_mask, 'Test': test_mask, 'All': model_data['ikn'].notnull()}
result = defaultdict(dict)
for cohort, mask in cohort_masks.items():
    df = model_data[mask]
    
    # All patients
    N = df['ikn'].nunique()
    get_ckd_count(df, result, key=(cohort, f'All patients (N={N})'))

    # Patients without CKD baseline
    mask = df['baseline_ckd'] < 60
    df = df[~mask]
    N = df['ikn'].nunique()
    get_ckd_count(df, result, key=(cohort, f'No Pre-treatment CKD (N={N})'))

result = pd.DataFrame(result)
result.to_csv(f'{output_path}/tables/ckd_summary.csv')
result

## Feature Characteristic

In [ ]:
df = prep.ohe.encode(model_data.copy(), verbose=False) # get original (non-normalized, non-imputed) data one-hot encoded
df = df[train_mask].drop(columns=['ikn'])
feature_summary(
    df, save_dir=f'{output_path}/tables', deny_old_survey=True, event_dates=prep.event_dates[train_mask]
).head(60)

# Train Models

## Spline Baseline Model

In [ ]:
trainer = PolynomialModelTrainer(X, Y, tag, output_path, base_col='baseline_eGFR', alg='SPLINE')
trainer.run(bayesopt=True, train=True, save=True)

In [ ]:
# save the model as a table
df = trainer.model_to_table(
    model=load_pickle(output_path, 'SPLINE'),
    base_vals=model_data['baseline_eGFR'],
    extra_info=model_data[['baseline_creatinine_value', 'next_eGFR']]
)
df.to_csv(f'{output_path}/SPLINE_model.csv')
df

## Main Models

In [ ]:
trainer = Trainer(X, Y, tag, output_path)
trainer.run(bayesopt=True, train=True, save_preds=True, algs=['LR', 'RF', 'XGB', 'NN'])

## ENS Model 
Find Optimal Ensemble Weights

In [ ]:
preds = load_pickle(f'{output_path}/preds', 'all_preds')
ensembler = Ensembler(X, Y, tag, output_path, preds)
ensembler.run(bayesopt=True, calibrate=True)

# Evaluate Models

In [ ]:
# setup the final prediction and labels
preds, labels = ensembler.preds, ensembler.labels
# Include the baseline models
preds.update(SimpleBaselineModel(model_data[['regimen', 'baseline_eGFR']], labels).predict())
preds.update(load_pickle(f'{output_path}/preds', 'SPLINE_preds'))

In [ ]:
evaluator = EvaluateClf(output_path, preds, labels)
scores = evaluator.get_evaluation_scores(display_ci=True, load_ci=True, save_ci=True)
scores

In [ ]:
scores.loc[['ENS', 'SPLINE']]

In [ ]:
evaluator.plot_curves(curve_type='pr', legend_loc='lower left', save=False)
evaluator.plot_curves(curve_type='roc', legend_loc='lower right', save=False)
evaluator.plot_curves(curve_type='pred_cdf', save=False) # cumulative distribution function of model prediction
evaluator.plot_calibs(legend_loc='upper left', save=False) 
# evaluator.plot_calibs(include_pred_hist=True, legend_loc='upper left', figsize=(12,28), padding={'pad_y1': 0.3, 'pad_y0': 3.0})

In [ ]:
for alg in ['SPLINE', 'ENS']:
    print('#' * 50 + f'\n# {alg}\n' + '#' * 50)
    evaluator.all_plots_for_all_targets(alg=alg)

# Post-Training Analysis

## Threshold Op Points

In [ ]:
pred_thresholds = np.arange(0.05, 0.51, 0.05)
perf_metrics = ['warning_rate', 'precision', 'recall', 'NPV', 'specificity']
thresh_df = evaluator.operating_points(pred_thresholds, alg='ENS', op_metric='threshold', perf_metrics=perf_metrics)
thresh_df

## Most Important Features/Feature Groups

In [ ]:
!python scripts/feat_imp.py --adverse-event CKD --output-path {output_path}

In [ ]:
# importance score is defined as the decrease in AUROC Score when feature value is randomly shuffled
importance_plot('ENS', evaluator.target_events, output_path, figsize=(6,15), top=10, importance_by='feature', padding={'pad_x0': 2.7})

In [ ]:
!python scripts/feat_imp.py --adverse-event CKD --output-path {output_path} --permute-group

In [ ]:
# importance score is defined as the decrease in AUROC Score when feature value is randomly shuffled
importance_plot('ENS', evaluator.target_events, output_path, figsize=(6,15), top=10, importance_by='group', padding={'pad_x0': 1.2})

## Performance on Subgroups

In [ ]:
custom_index = [
    ('Regimen', 'CISP RT-W (21.4%)'),
    ('Regimen', 'CISPETOP 3D (12.6%)'),
    ('Regimen', 'CISP RT (11.7%)'),
    ('Topography ICD-0-3', 'Bronchus and Lung (31.7%)'),
    ('Topography ICD-0-3', 'Cervix Uteri (8.7%)'),
    ('Topography ICD-0-3', 'Bladder (5.6%)'),
    ('CKD Prior to Treatment', 'Has CKD (6.3%)'),
    ('CKD Prior to Treatment', 'No CKD (93.7%)'),
    ('Diabetes Status', 'Diabetic (15.7%)'),
    ('Diabetes Status', 'Non-Diabetic (84.3%)'),
    ('Hypertension Status', 'Hypertensive (48.0%)'),
    ('Hypertension Status', 'Non-Hypertensive (52.0%)')
]

### ENS

In [ ]:
subgroups = ['all', 'age', 'sex', 'immigrant', 'language', 'arrival', 'income', 'area_density', 'ckd', 'regimen', 'cancer_location', 'comorbidity']
subgroup_performance = evaluator.get_perf_by_subgroup(
    model_data, subgroups=subgroups, pred_thresh=0.1, alg='ENS', fname='ENS_subgroup_performance',
    display_ci=True, load_ci=True, filename_ci='ENS_bootstrapped_subgroup_scores'
)
subgroup_performance

In [ ]:
subgroup_performance = pd.read_csv(f'{output_path}/tables/ENS_subgroup_performance.csv', index_col=[0,1], header=[0,1])
subgroup_performance.index = pd.MultiIndex.from_tuples(subgroup_performance.index[:18].tolist() + custom_index)
padding = {'pad_y0': 1.2, 'pad_x1': 2.7, 'pad_y1': 0.2}
subgroups = ['Entire Test Cohort', 'Age', 'Sex', 'Immigration', 'Area of Residence', 'Regimen', 'Topography ICD-0-3', 'Diabetes Status', 'Hypertension Status']
subgroup_performance_plot(
    subgroup_performance, target_event='CKD', subgroups=subgroups, padding=padding,
    figsize=(12,30), save_dir=f'{output_path}/figures/subgroup_perf/ENS', xtick_rotation=75
)

### SPLINE

In [ ]:
subgroups = ['all', 'age', 'sex', 'immigrant', 'language', 'arrival', 'income', 'area_density', 'ckd', 'regimen', 'cancer_location', 'comorbidity']
subgroup_performance = evaluator.get_perf_by_subgroup(
    model_data, subgroups=subgroups, pred_thresh=0.1, alg='SPLINE', fname='SPLINE_subgroup_performance',
    display_ci=True, load_ci=True, filename_ci='SPLINE_bootstrapped_subgroup_scores'
)
subgroup_performance

In [ ]:
subgroup_performance = pd.read_csv(f'{output_path}/tables/SPLINE_subgroup_performance.csv', index_col=[0,1], header=[0,1])
subgroup_performance.index = pd.MultiIndex.from_tuples(subgroup_performance.index[:18].tolist() + custom_index)
padding = {'pad_y0': 1.2, 'pad_x1': 2.7, 'pad_y1': 0.2}
subgroups = ['Entire Test Cohort', 'Age', 'Sex', 'Immigration', 'Area of Residence', 'Regimen', 'Topography ICD-0-3', 'Diabetes Status', 'Hypertension Status']
subgroup_performance_plot(
    subgroup_performance, target_event='CKD', subgroups=subgroups, padding=padding,
    figsize=(12,30), save_dir=f'{output_path}/figures/subgroup_perf/SPLINE', xtick_rotation=75
)

## Decision Curve Plot

In [ ]:
result = evaluator.plot_decision_curves('ENS')
result['CKD'].tail(n=100)

In [ ]:
result = evaluator.plot_decision_curves('SPLINE')
result['CKD'].tail(n=100)

In [ ]:
get_hyperparameters(output_path)

## Prediction vs Baseline Plots

In [ ]:
preds = load_pickle(f'{output_path}/preds', 'SPLINE_preds')
pred_ci = load_pickle(f'{output_path}/preds', 'SPLINE_preds_ci')
baseline_evaluator = EvaluateBaselineModel(
    model_data['baseline_eGFR'][test_mask], preds, labels, output_path, pred_ci=pred_ci
)
target_events = Y.columns
fig, ax = plt.subplots(figsize=(6,6))
for target_event in Y.columns:
    baseline_evaluator.plot_pred_vs_base(ax, alg='SPLINE', target_event=target_event, split='Test', open_top_right=False)
plt.savefig(f'{output_path}/figures/baseline/SPLINE_pred_vs_baseline.svg', format='svg', bbox_inches='tight', dpi=300)

## Score Bar Plot

In [ ]:
score_plot(scores, Y.columns, output_path, metric='AUROC', ylim=(0.49, 1.01))
score_plot(scores, Y.columns, output_path, metric='AUPRC', ylim=(0, 1.01))

# Scratch Notes

## Missingness By Splits

In [ ]:
from src.utility import get_nmissing_by_splits

In [ ]:
missing = get_nmissing_by_splits(model_data, ensembler.labels)
missing.sort_values(by=(f'Test (N={sum(test_mask)})', 'Missing (N)'), ascending=False)